In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pylab as plt
import math

filepath = 'data/heart.csv'
df = pd.read_csv(filepath)

# Introduction
This project explores the machine learning pipeline by working on [Heart Failure Data](https://www.kaggle.com/fedesoriano/heart-failure-prediction), found on Kaggle. The target variable we classify is HeartDisease, a binary datatype where 1 indicates the person has Heart Disease, and 0 (normal) otherwise. Heart disease is the leading cause of death in the world, and machine learning models that can predict which persons are at a high risk for heart disease early can help spur action to increase the likelihood of positive health outcomes for those users, in addition to encouraging persons who might not be at risk currently to make cautionary lifestyle changes. This dataset comprises 918 persons, recording 12 observations each. This datset is already described as follows (the annotation (CTS) means that attribute is continuous, while (CAT) indicates the attribute is categorical, and (ORD) indicates ordinal attribute):

Age: age of the patient (years) (CTS)

Sex: sex of the patient (M: Male, F: Female) (CAT)

ChestPainType: chest pain type (TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic)

RestingBP: resting blood pressure (mm Hg)

Cholesterol: serum cholesterol (mm/dl)

FastingBS: fasting blood sugar (1: if FastingBS > 120 mg/dl, 0: otherwise)

RestingECG: resting electrocardiogram results (Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria)

MaxHR: maximum heart rate achieved (Numeric value between 60 and 202)

ExerciseAngina: exercise-induced angina (Y: Yes, N: No)

Oldpeak: oldpeak = ST (Numeric value measured in depression)

ST_Slope: the slope of the peak exercise ST segment (Up: upsloping, Flat: flat, Down: downsloping)

HeartDisease: output class (1: heart disease, 0: Normal)




In [2]:
print("Rows: "+ str(df.shape[0]))
print("Columns: "+ str(df.shape[1]))
cols = list(df.columns)
for col in cols:
    df1 = df[col].dropna()
    print(f"dtype of {col} is: {df1.dtype}")

Rows: 918
Columns: 12
dtype of Age is: int64
dtype of Sex is: object
dtype of ChestPainType is: object
dtype of RestingBP is: int64
dtype of Cholesterol is: int64
dtype of FastingBS is: int64
dtype of RestingECG is: object
dtype of MaxHR is: int64
dtype of ExerciseAngina is: object
dtype of Oldpeak is: float64
dtype of ST_Slope is: object
dtype of HeartDisease is: int64


In [4]:
y = df["HeartDisease"]
y.nunique()

2

### References
fedesoriano. (September 2021). Heart Failure Prediction Dataset. Retrieved 6 October 2021 from https://www.kaggle.com/fedesoriano/heart-failure-prediction.